In [122]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

import os
from pathlib import Path

## Tomato Predictor - An introduction
The idea of this project is to learn about:

1. Convolutional Neural Networks (CNNs)
    - Convolution and pooling
    - Common arquitectures - LeNet, AlexNet, VGG, ResNet

2. Image Preprocessing
    - Normalization and standarization of pixels
    - Data Augmentation: from the same sample, generate more images
    - Noise reduction and constrast enhancement

3. Evaluation metrics
    - Precission, recall, F1-score
    - Confusion matrix
    - ROC and AUC curve

4. Optimization and regularization
    - Optimization algorithm - SGC, Adam, RMSprop
    - Techniques to avoid overfitting - dropout, batch normalization, early stopping

## Data

### Extracting data from directories

In [139]:
from pathlib import Path

route_train = 'content/ieee-mbl-cls/train'
route_test = 'content/ieee-mbl-cls/val'
 
dynamic_route_train = "{}/{}".format(route_train, "{}")
dynamic_route_test = "{}/{}".format(route_test, "{}")

labels = []
allowed_extension = ".jpg"

def extract_labels(route: str) -> None:
    pathlist = Path(route).iterdir()
    for path in pathlist:
        # because path is object not string
        label = str(path.name)   
        labels.append(label)

def convert_data_to_df(dynamic_route):
    images = {}  # dict with "image_key_name": "label"

    for label in labels:
        path = Path(dynamic_route.format(label))

        for img_path in path.iterdir():
            ext = img_path.suffix.lower()
            if ext != allowed_extension:
                print(f"Ignorando: {img_path.name} ({ext})")
                continue
            
            images[img_path.name] = img_path.name[0]

    df = pd.DataFrame({
        "name": list(images.keys()),
        "label": list(images.values()), 
    })
    
    return df
    
extract_labels(route_train)

In [136]:
df = convert_data_to_df(dynamic_route_train)
df = df.sample(frac=1).reset_index(drop=True) # shuffle the dataframe
df.head()

Starting conversion with allowed_extension=.jpg
Processing directory: content/ieee-mbl-cls/train/Unripe
Found file: u (1050).jpg with extension .jpg
Adding to images dict: u (1050).jpg
Found file: u (481).jpg with extension .jpg
Adding to images dict: u (481).jpg
Found file: u (1400).jpg with extension .jpg
Adding to images dict: u (1400).jpg
Found file: u (878).jpg with extension .jpg
Adding to images dict: u (878).jpg
Found file: u (340).png with extension .png
Ignorando: u (340).png (.png)
Found file: u (205).png with extension .png
Ignorando: u (205).png (.png)
Found file: u (897).jpg with extension .jpg
Adding to images dict: u (897).jpg
Found file: u (1115).jpg with extension .jpg
Adding to images dict: u (1115).jpg
Found file: u (194).jpg with extension .jpg
Adding to images dict: u (194).jpg
Found file: u (1396).jpg with extension .jpg
Adding to images dict: u (1396).jpg
Found file: u (747).jpg with extension .jpg
Adding to images dict: u (747).jpg
Found file: u (317).jpg with 

,name,label
0,r (1026).jpg,r
1,o (765).jpg,o
2,r (1007).jpg,r
3,r (1477).jpg,r
4,r (1164).jpg,r


In [137]:
df_test = convert_data_to_df(dynamic_route_test)
df_test = df_test.sample(frac=1).reset_index(drop=True) # shuffle the dataframe
df_test.head()

Starting conversion with allowed_extension=.jpg
Processing directory: content/ieee-mbl-cls/val/Unripe
Found file: u (57).jpg with extension .jpg
Adding to images dict: u (57).jpg
Found file: u (593).jpg with extension .jpg
Adding to images dict: u (593).jpg
Found file: u (950).jpg with extension .jpg
Adding to images dict: u (950).jpg
Found file: u (511).jpg with extension .jpg
Adding to images dict: u (511).jpg
Found file: u (638).jpg with extension .jpg
Adding to images dict: u (638).jpg
Found file: u (792).jpg with extension .jpg
Adding to images dict: u (792).jpg
Found file: u (229).png with extension .png
Ignorando: u (229).png (.png)
Found file: u (989).jpg with extension .jpg
Adding to images dict: u (989).jpg
Found file: u (618).jpg with extension .jpg
Adding to images dict: u (618).jpg
Found file: u (634).jpg with extension .jpg
Adding to images dict: u (634).jpg
Found file: u (233).png with extension .png
Ignorando: u (233).png (.png)
Found file: u (788).jpg with extension .j

,name,label
0,r (566).jpg,r
1,r (2496).jpg,r
2,d (389).jpg,d
3,u (270).jpg,u
4,r (956).jpg,r


### From images to pixels

Our images are a 256px x 256px. Then, we have adjust our CNN for this. However, we first have to extract the pixel information of the image in order to be able to process the data in the neural network.

In [140]:
import pandas as pd
from PIL import Image
from numpy import asarray

def add_pixels_image_data(df, route, index, name, target_size=(256, 256)):
    """
    For the row at the specified index, it will append the image's pixel data after processing it.
    The image will be resized and normalized before being added to the DataFrame.
    """
    label = ''
    first = name[0]
    
    if first == 'r':
        first = "Ripe"
    elif first == "o":
        first = "Old"
    elif first == "u":
        first = "Unripe"
    else:
        first = "Damaged"

    path = f"{route}/{first}/{name}"

    image = Image.open(path)
    print(f"Format: {image.format}, pixels: {image.size}, mode: {image.mode}\n")    
    image = image.resize(target_size)

    numpydata = asarray(image)
    numpydata = numpydata.astype('float32')

    flattened_image = numpydata.flatten()

    if 'image' not in df.columns:
        df['image'] = pd.Series([None] * len(df))

    df.at[index, 'image'] = flattened_image / 255 # normalise data [0, 1]

    return df

In [ ]:
for index, row in df.iterrows():
    df = add_pixels_image_data(df, route_train, index, row['name'])

In [142]:
df.head()

,name,label,image
0,r (1026).jpg,r,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,o (765).jpg,o,"[0.7607843, 0.76862746, 0.7647059, 0.7607843, ..."
2,r (1007).jpg,r,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,r (1477).jpg,r,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,r (1164).jpg,r,"[0.015686275, 0.043137256, 0.015686275, 0.0549..."


In [143]:
df.shape

(5832, 3)

In [144]:
df.at[6, "image"].shape

(196608,)

## Basic Neural Network

### How to determine the number of nodes and layers?

Funny enough, I found this in StackOverflow: [Publication](https://stackoverflow.com/questions/35520587/how-to-determine-the-number-of-layers-and-nodes-of-a-neural-network).

---
For the **layers**, just keep adding layers until the test error does not improve anymore.

For the **nodes**, you should have one node per feature (in the input layer), which makes sense.

In [131]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D

In [155]:
for i, img in enumerate(df['image']):
    if img.shape != (196608,):
        print(f"Error en la imagen {i}: {img}")

Error en la imagen 19: [0.54901963 0.5294118  0.5137255  ... 0.654902   0.6627451  1.        ]
Error en la imagen 65: [0.5529412  0.5411765  0.52156866 ... 0.6431373  0.654902   1.        ]
Error en la imagen 182: [1. 1. 1. ... 1. 1. 1.]
Error en la imagen 431: [1. 1. 1. ... 1. 1. 1.]
Error en la imagen 514: [0.654902   0.67058825 0.7137255  ... 0.54509807 0.54509807 1.        ]
Error en la imagen 599: [0.6392157  0.6431373  0.65882355 ... 0.5137255  0.50980395 1.        ]
Error en la imagen 721: [0.6509804  0.6509804  0.65882355 ... 0.5254902  0.5137255  1.        ]
Error en la imagen 870: [0.5411765  0.5254902  0.5137255  ... 0.6509804  0.65882355 1.        ]
Error en la imagen 872: [0.76862746 0.7529412  0.70980394 ... 0.7254902  0.7058824  1.        ]
Error en la imagen 911: [0.5529412  0.54901963 0.53333336 ... 0.6745098  0.6901961  1.        ]
Error en la imagen 929: [0.5294118 0.5137255 0.5019608 ... 0.6431373 0.6627451 1.       ]
Error en la imagen 954: [1. 1. 1. ... 1. 1. 1.]


In [60]:
df.at[5561, 'name']

'u (94).jpg'

In [55]:
df.at[45, 'image'].shape

(196608,)

In [ ]:
model = Sequential()
model.add(Dense(196608, input_shape=(196608,), activation='relu')) # hidden layer
model.add(Dense(4, activation='softmax')) # softmax bcs we want to obtain the max prob of the four labels


model.summary()
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# training the model for 10 epochs
model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_test, Y_test))